## Botnet discovery with unknown K:
All accounts are treated uniformly; background = all *other* accounts (leave-two-out).
We compute Impostors-style AV similarities and detect communities via Louvain modularity.

* https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.louvain.louvain_communities.html
* https://ceur-ws.org/Vol-1179/CLEF2013wn-PAN-Seidman2013.pdf
* https://arxiv.org/abs/1810.08473


In [1]:
import re, random, numpy as np, pandas as pd
from typing import Dict, List, Tuple
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import itertools
import networkx as nx
from networkx.algorithms.community import louvain_communities

In [2]:
df = pd.read_csv("./PhDBotnetsDB/User_Tweet/_outputs/user_tweets_full_enriched.csv")

def get_tweets_from_user(user_id: int) -> List[str]:
    return df.loc[(df["user_id"] == user_id) & (df["is_retweet"] == False), "tweet_text"].dropna().astype(str).tolist()

accounts: Dict[str, List[str]] = {
    "acc_A": get_tweets_from_user(21479334), # cresci17 - Traditional Spambots - 4
    "acc_B": get_tweets_from_user(1544624887), # starwars
    "acc_C": get_tweets_from_user(1585571839), # starwars
    "acc_D": get_tweets_from_user(21674676), # cresci17 - Traditional Spambots - 4
    "acc_E": get_tweets_from_user(1546564405), # starwars
    "acc_F": get_tweets_from_user(1587209611), # starwars
    "acc_G": get_tweets_from_user(1552496288), # starwars
    "acc_H": get_tweets_from_user(22728442), # cresci17 - Traditional Spambots - 4
    "acc_I": get_tweets_from_user(22759829), # cresci17 - Traditional Spambots - 4
    "acc_J": get_tweets_from_user(22578323), # cresci17 - Traditional Spambots - 4
    "acc_K": get_tweets_from_user(74793689), # cresci17 - Traditional Spambots - 4
    "acc_L": get_tweets_from_user(38095383), # cresci17 - Traditional Spambots - 4
    "acc_M": get_tweets_from_user(1557300746), # starwars
    "acc_N": get_tweets_from_user(1559362050), # starwars
}

C:\Users\felip\AppData\Local\Temp\ipykernel_23972\1746022432.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./PhDBotnetsDB/User_Tweet/_outputs/user_tweets_full_enriched.csv")


In [10]:
df.loc[df["carpeta_origen"].eq("JournalistAttackBrianKrebs"), "user_id"].value_counts().iloc[100:120]

user_id
725325656             1103
1401156210            1073
473212705             1021
45571859               975
500713601              935
82209264               929
57019818               928
19512396               928
276803336              893
541492320              892
3273475567             875
111094423              851
2423506384             787
1655215466             766
4645681828             734
105222131              730
54260807               700
851226858280505345     683
227184175              669
226040131              611
Name: count, dtype: int64

In [16]:
df["carpeta_origen"].value_counts()
#print(df[df["carpeta_origen"] == "Cresci2015-INT"]["user_id"].value_counts())

carpeta_origen
Cresci2017-SocialSpambots-3         1408728
Cresci2017-SocialSpambots-1         1269264
Cresci2017-TraditionalSpambots-3     791735
JournalistAttackBrianKrebs           304017
Cresci2017-TraditionalSpambots-1     185533
Cresci2015-TWT                       156009
Cresci2017-TraditionalSpambots-4      60600
StarWarsBotnet                        26971
Cresci2015-INT                         3315
Cresci2015-FSF                          695
Name: count, dtype: int64

## Normalization & chunking

Normalization removes URL/mention noise and standardizes casing/spacing.
Chunking creates *equal-sized* text blocks so similarity isn't dominated by account length.
This is common in AV to stabilize short/noisy texts and prevent length/template bias.

In [4]:
URL_RE        = re.compile(r'https?://\S+|www\.\S+')
MENTION_RE    = re.compile(r'@\w+')
WHITESPACE_RE = re.compile(r'\s+')

def normalize_tweet(t: str, keep_hashtags: bool = True) -> str:
    """
    Lowercase, strip URLs and @mentions, collapse whitespace.
    Keep hashtags by removing the '#' (style usage can be informative).
    """
    t = t.lower()
    t = URL_RE.sub(" ", t)
    t = MENTION_RE.sub(" ", t)
    if keep_hashtags:
        t = t.replace("#", "")
    t = t.replace("&amp;", "&")
    t = WHITESPACE_RE.sub(" ", t).strip()
    return t

def unique_norm_texts(texts: List[str]) -> List[str]:
    """
    Deduplicate normalized tweets so repeated boilerplate doesn't overweight the model.
    (We don't hardcode templates; we just avoid exact duplicate rows after normalization.)
    """
    seen, uniq = set(), []
    for s in texts:
        n = normalize_tweet(s)
        if n and n not in seen:
            seen.add(n); uniq.append(n)
    return uniq

def make_char_chunks(texts: List[str], target_chars: int = 5000, max_chunks: int = 12) -> List[str]:
    """
    Build ~equal-sized character chunks (≈target_chars).
    This balances accounts with many vs. few tweets and yields multiple comparable samples per account.
    """
    msgs = unique_norm_texts(texts)
    if not msgs:
        return []
    chunks, cur, cur_len = [], [], 0
    for m in msgs:
        cur.append(m); cur_len += len(m) + 1
        if cur_len >= target_chars:
            chunks.append(" ".join(cur))
            cur, cur_len = [], 0
            if len(chunks) >= max_chunks:
                break
    if cur and len(chunks) < max_chunks:
        chunks.append(" ".join(cur))
    return chunks if chunks else [" ".join(msgs)]


## Vectorization (TF–IDF over char n-grams)
Char n-grams are strong, topic-light style signals for short/informal texts (tweets),
widely used in AV & PAN tasks. We also guard TF–IDF with min_df/max_df to drop too-rare or too-common grams.
scikit-learn semantics: min_df/max_df can be counts or proportions; we auto-tune to avoid conflicts.


In [5]:
def fit_vectorizer(
    acc_texts: Dict[str, List[str]],
    ngram_range: Tuple[int,int]=(3,5),
    max_features: int = 200_000,
    min_df=None, max_df=None, sublinear_tf: bool = True,
):
    docs, ids = [], []
    per_acc_counts = {}
    for aid, texts in acc_texts.items():
        chunks = make_char_chunks(texts, target_chars=5000, max_chunks=12)
        per_acc_counts[aid] = len(chunks)
        for ch in chunks:
            docs.append(ch); ids.append(aid)

    n_docs = len(docs)
    if n_docs < 2:
        raise ValueError(f"Too few documents/chunks ({n_docs}). Increase data or lower target_chars.")

    # Auto-safe DF thresholds to respect sklearn's constraint: max_df_docs >= min_df_docs
    user_min, user_max = min_df, max_df
    if min_df is None: min_df = max(1, int(0.01 * n_docs)) 
    if max_df is None: max_df = 0.90                         

    max_df_docs = int(max_df * n_docs) if isinstance(max_df, float) else int(max_df)
    min_df_docs = int(min_df) if isinstance(min_df, int) else int(min_df * n_docs)

    if max_df_docs < min_df_docs:
        min_df_docs = max(1, max_df_docs)
        if isinstance(user_min, float): min_df = max(user_min, min_df_docs / n_docs)
        else:                           min_df = min_df_docs

    max_df_docs = int(max_df * n_docs) if isinstance(max_df, float) else int(max_df)
    min_df_docs = int(min_df) if isinstance(min_df, int) else int(min_df * n_docs)
    if max_df_docs < min_df_docs:
        if isinstance(max_df, float): max_df = min(0.99, (min_df_docs + 1) / n_docs)
        else:                         max_df = min_df_docs + 1

    print(f"[fit_vectorizer] n_docs={n_docs} chunks_per_account={per_acc_counts}  min_df={min_df}  max_df={max_df}")

    vec = TfidfVectorizer(
        analyzer="char", ngram_range=ngram_range,
        min_df=min_df, max_df=max_df,
        max_features=max_features,
        norm="l2", sublinear_tf=sublinear_tf, 
    )
    X = vec.fit_transform(docs)

    account_vecs = {}
    for i, aid in enumerate(ids):
        account_vecs.setdefault(aid, []).append(X[i])

    n_features = len(vec.get_feature_names_out())
    return vec, account_vecs, n_features

## Impostors-style AV (chunked + symmetric)

The Impostors method answers "same author?" by asking if A–B similarity
repeatedly beats A–(random background) across many random feature subspaces.
We run it symmetrically (A→B and B→A) to reduce anchor bias; this matches
good AV practice on short texts.

In [6]:
def set_random_seed(seed=42):
    random.seed(seed); np.random.seed(seed)

def _random_feature_mask(n_features: int, keep_frac: float) -> np.ndarray:
    """Sample a random subset of features (columns) to form a subspace per trial."""
    k = max(1, int(n_features * keep_frac))
    mask = np.zeros(n_features, dtype=bool)
    idx = np.random.choice(n_features, size=k, replace=False)
    mask[idx] = True
    return mask

def _masked_cosine(a, b, feat_mask: np.ndarray) -> float:
    """Cosine similarity in the chosen subspace (1 x d sparse rows)."""
    return float(cosine_similarity(a[:, feat_mask], b[:, feat_mask])[0, 0])

def _pick_chunk(vecs: List) -> np.ndarray:
    """Uniformly sample one chunk vector for this account (varies across trials)."""
    if not vecs: raise ValueError("Empty chunk list for account.")
    return vecs[np.random.randint(len(vecs))]

def impostors_score_chunks(
    A_chunks: List, B_chunks: List, background_accounts: List[List],
    n_features: int, n_trials: int = 400, feat_frac: float = 0.5, bg_accounts_per_trial: int = 25
) -> float:
    """
    Fraction of trials where sim(A,B) > max sim(A, any sampled background account),
    using a fresh random feature subspace each trial.
    """
    poolN = len(background_accounts)
    if poolN == 0:
        return 0.0
    wins = 0
    k = min(bg_accounts_per_trial, poolN)
    for _ in range(n_trials):
        mask = _random_feature_mask(n_features, feat_frac)
        vA = _pick_chunk(A_chunks)
        vB = _pick_chunk(B_chunks)
        sim_AB = _masked_cosine(vA, vB, mask)
        idxs = np.random.choice(poolN, size=k, replace=False)
        sims_AI = [_masked_cosine(vA, _pick_chunk(background_accounts[j]), mask) for j in idxs]
        if sim_AB > max(sims_AI, default=-1.0):
            wins += 1
    return wins / n_trials

def symmetric_impostors_score(
    A_chunks: List, B_chunks: List, background_accounts: List[List],
    n_features: int, n_trials: int = 400, feat_frac: float = 0.5, bg_accounts_per_trial: int = 25
) -> float:
    """Average of A→B vs background and B→A vs background."""
    s1 = impostors_score_chunks(A_chunks, B_chunks, background_accounts,
                                n_features, n_trials, feat_frac, bg_accounts_per_trial)
    s2 = impostors_score_chunks(B_chunks, A_chunks, background_accounts,
                                n_features, n_trials, feat_frac, bg_accounts_per_trial)
    return 0.5 * (s1 + s2)

## Build all-pairs similarity (leave-two-out background)

We treat *all* accounts uniformly. For each pair (A,B),
the background is "all other accounts" (leave-two-out).
This removes the need for a separate impostor list and keeps the setting fully unsupervised.

In [7]:
def build_similarity_matrix(
    ids: List[str],
    account_chunk_vectors: Dict[str, List],  # id -> [chunk vectors]
    n_features: int,
    n_trials: int = 800, feat_frac: float = 0.45, bg_accounts_per_trial: int = 40, seed: int = 42
) -> np.ndarray:
    np.random.seed(seed)
    n = len(ids)
    S = np.eye(n, dtype=float) 

    chunks = {bid: account_chunk_vectors[bid] for bid in ids}

    for i, j in itertools.combinations(range(n), 2):
        A_id, B_id = ids[i], ids[j]
        A_chunks, B_chunks = chunks[A_id], chunks[B_id]

        # Background = all other accounts (leave two out)
        background = [chunks[k] for k in ids if k not in (A_id, B_id)]

        s = symmetric_impostors_score(
            A_chunks, B_chunks, background, n_features,
            n_trials=n_trials, feat_frac=feat_frac, bg_accounts_per_trial=bg_accounts_per_trial
        )
        S[i, j] = S[j, i] = s
    return S


## Community detection (unknown K) via Louvain
Turn S into a graph (edge if similarity >= tau) and find communities that maximize modularity.
We sweep tau and keep the partition with the best modularity Q (NetworkX provides both).
`next implementation: Leiden`
Note: Leiden improves Louvain (connectedness & quality); consider it for larger graphs. 



In [8]:
def cluster_with_louvain(S: np.ndarray, ids: List[str], tau: float = 0.65, resolution: float = 1.0, seed: int = 42):
    """
    Threshold S at tau, build weighted graph, detect communities with Louvain.
    Returns (clusters_dict, modularity_Q).
    """
    G = nx.Graph()
    G.add_nodes_from(ids)
    for i in range(len(ids)):
        for j in range(i+1, len(ids)):
            w = float(S[i, j])
            if w >= tau:
                G.add_edge(ids[i], ids[j], weight=w)

    if G.number_of_edges() == 0:
        return {}, 0.0

    # Louvain communities (greedy modularity optimization) — returns sets of nodes
    comms = louvain_communities(G, weight="weight", resolution=resolution, seed=seed)  

    # Modularity score Q = quality of the partition (higher = clearer community structure)
    from networkx.algorithms.community import modularity
    Q = modularity(G, comms, weight="weight") 

    clusters = {i: sorted(list(c)) for i, c in enumerate(comms)}
    return clusters, Q

def sweep_tau_for_best_modularity(S: np.ndarray, ids: List[str], taus=np.linspace(0.58, 0.76, 5), resolution: float = 1.0):
    """
    Try several thresholds and keep the partition with highest modularity Q.
    This removes the need to guess a single tau value up front.
    """
    best = {"tau": None, "Q": -1, "clusters": None}
    for tau in taus:
        clusters, Q = cluster_with_louvain(S, ids, tau=tau, resolution=resolution)
        if Q is not None and Q > best["Q"]:
            best = {"tau": tau, "Q": Q, "clusters": clusters}
    return best

## Demo

In [ ]:
set_random_seed(42)

vec, account_chunk_vectors, n_features = fit_vectorizer(
    accounts, ngram_range=(3,5), min_df=None, max_df=None, sublinear_tf=True, max_features=120_000
)

ids_to_cluster = list(accounts.keys())
S = build_similarity_matrix(
    ids_to_cluster, account_chunk_vectors, n_features,
    n_trials=150, feat_frac=0.40, bg_accounts_per_trial=25, seed=42
)

print("IDs:", ids_to_cluster)
#print("Similarity matrix (rounded):\n", np.round(S, 3))

[fit_vectorizer] n_docs=14 chunks_per_account={'acc_A': 1, 'acc_B': 1, 'acc_C': 1, 'acc_D': 1, 'acc_E': 1, 'acc_F': 1, 'acc_G': 1, 'acc_H': 1, 'acc_I': 1, 'acc_J': 1, 'acc_K': 1, 'acc_L': 1, 'acc_M': 1, 'acc_N': 1}  min_df=1  max_df=0.9
IDs: ['acc_A', 'acc_B', 'acc_C', 'acc_D', 'acc_E', 'acc_F', 'acc_G', 'acc_H', 'acc_I', 'acc_J', 'acc_K', 'acc_L', 'acc_M', 'acc_N']
Similarity matrix (rounded):
 [[1.    0.    0.    1.    0.    0.    0.    0.086 0.    0.033 0.    0.471
  0.    0.   ]
 [0.    1.    0.065 0.    0.    0.001 0.421 0.    0.    0.    0.    0.
  0.16  0.   ]
 [0.    0.065 1.    0.    0.003 0.001 0.011 0.    0.    0.    0.    0.
  0.594 0.002]
 [1.    0.    0.    1.    0.    0.    0.    0.    0.059 0.    0.017 0.
  0.    0.   ]
 [0.    0.    0.003 0.    1.    0.003 0.    0.    0.    0.    0.    0.
  0.001 0.498]
 [0.    0.001 0.001 0.    0.003 1.    0.227 0.    0.    0.    0.    0.
  0.236 0.063]
 [0.    0.421 0.011 0.    0.    0.227 1.    0.    0.    0.    0.    0.
  0.117 0.1

In [10]:
best = sweep_tau_for_best_modularity(S, ids_to_cluster, taus=np.linspace(0.58, 0.76, 5), resolution=1.0)
print(f"\n[Louvain] best_tau={best['tau']:.2f}  modularity(Q)={best['Q']:.3f}")
for cid, members in (best["clusters"] or {}).items():
    print(f"  Botnet {cid}: {members}")


[Louvain] best_tau=0.62  modularity(Q)=0.499
  Botnet 0: ['acc_A', 'acc_D']
  Botnet 1: ['acc_B']
  Botnet 2: ['acc_C']
  Botnet 3: ['acc_E']
  Botnet 4: ['acc_F']
  Botnet 5: ['acc_G']
  Botnet 6: ['acc_H']
  Botnet 7: ['acc_I']
  Botnet 8: ['acc_J']
  Botnet 9: ['acc_K']
  Botnet 10: ['acc_L']
  Botnet 11: ['acc_M', 'acc_N']
